# Email Scraper

import deps

In [ ]:
import grequests
from bs4 import BeautifulSoup
import re
import json
from urllib.parse import urlparse, unquote

read list of URLs to scrape from `in.txt`

In [ ]:
input_file = 'in.json'

with open('./in/' + input_file) as f:
    urls = json.load(f)

urls

scrape emails from anchor tags with mailto links

In [ ]:
def is_valid_email_address(string):
  email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b')
  return email_pattern.fullmatch(string) != None

def extract_emails(response):
  
  # Parse the HTML content
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find all 'mailto:' links
  anchors_with_mailto_links = soup.find_all('a', href=re.compile(r'^mailto:'))

  # Extract email addresses from 'mailto:' links
  emails = set()
  for anchor in anchors_with_mailto_links:
    mailto_link = anchor['href']
    parsed_link = urlparse(mailto_link, "mailto")
    
    email = parsed_link.path
    email = unquote(email)
    email = email.strip()
    
    if is_valid_email_address(email):
      emails.add(email)

  return emails

In [ ]:
requests = (grequests.get(u) for u in urls)

email_addresses = set()
for response in grequests.imap(requests, size=10):
  email_addresses.update(extract_emails(response))

In [ ]:
print(f"Total email addresses: {len(email_addresses):0}")
email_addresses

write scraped email address to file

In [ ]:
output_file = 'out.txt'

with open('./out/' + output_file, 'w+') as f:
    f.writelines(", ".join(email_addresses))